
# 预测指导产量，最大化总利润：
成本损失，利润损失。
如果指导多生产，滞销，有成本损失，如果少生产，有利润损失。

例子1：成本和利润损失，对y的影响平等。
loss = tf.reduce_mean(tf.square(y_ - y))
w1: [[ 1.01073086]
[ 0.99547988]]
#总和：2.005


例子2.1：成本和利润损失，对y的影响不平等。有权重。单纯MSE就不能衡量利润了。本例系数：成本损失1,利润损失9.

COST = 1
PROFIT = 9
loss = tf.reduce_sum(tf.where(tf.greater(y,y_), (y-y_)*COST,(y_-y)*PROFIT))
w1: [[ 1.12403703]
 [ 0.9461537 ]]
最终得到的w1,总和2.070
和前者明显不同，倾向于预测更大的总和。
结果：w1总的更大，倾向于高产量。

例子2.2：成本和利润损失，对y的影响不平等。有权重。单纯MSE就不能衡量利润了。本例系数：成本损失1,利润损失9.
COST = 9
PROFIT = 1
结果：w1总的更小，不过最后一次可能有点过小了（Momentum造成的，SGD不会）。
w1: [[ 0.95781273]
 [ 0.97853577]]


例子2.3：成本和利润损失，对y的影响不平等。有权重。单纯MSE就不能衡量利润了。本例系数：成本损失5,利润损失5.
原则上，结果应该和例子1一致
w1: [[ 1.0408988 ]
 [ 1.07546258]]
 2.115


区别主要是loss，是对y预测多寡的惩罚不同，x1和x2没有具体含义。


In [1]:
import tensorflow as tf
import numpy as np

In [7]:
BATCH_SIZE = 64
DATA_SIZE = 256
SEED = 23455
COST = 5
PROFIT = 5

In [8]:
#X每个特征均值0.5,Y均值1
#X不是随机的么，不是每个数值均值1?是0~1均匀分布，均值0.5
rdm = np.random.RandomState(SEED)
X = rdm.rand(DATA_SIZE,2)#shape，DATA_SIZE个样本，2个属性
X2 = rdm.rand(DATA_SIZE,5)#shape，DATA_SIZE个样本，2个属性
Y = [[x1 + x2 + rdm.rand()/10.0-0.05] for (x1, x2) in X]#加入噪音,均值0
# print(Y)
with tf.Session() as sess:
    print(sess.run(tf.reduce_mean(X)))#也可以借助tf计算均值
    print(sess.run(tf.reduce_mean(X2)))#也可以借助tf计算均值
    print(sess.run(tf.reduce_mean(Y)))#也可以借助tf计算均值

0.49517288969
0.495503961307
0.994177


In [9]:
x = tf.placeholder(tf.float32, shape=(None,2))
y_ = tf.placeholder(tf.float32, shape=(None,1))
w1 = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)

#两种loss，一个是对等MSE，一个是有权重的
# loss = tf.reduce_mean(tf.square(y_ - y))

#如果y比y_大，预测多了，损失成本，系数COST，预测少了，系数PROFIT
loss = tf.reduce_sum(tf.where(tf.greater(y,y_), (y-y_)*COST,(y_-y)*PROFIT))


#train_op = tf.train.MomentumOptimizer(0.001,0.9).minimize(loss)#本例尽量不要用momentum，感觉影响有点大
train_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [10]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    STEPS = 20000
    print('w1:',sess.run(w1))
    for i in range(STEPS):
        start = (i*BATCH_SIZE)%DATA_SIZE
        end = start+BATCH_SIZE
        sess.run(train_op,feed_dict={x:X[start:end], y_:Y[start:end]})
        if i % 500 == 0:
            print('w1:',sess.run(w1))#经过训练，w1的两个变量都接近1，因为X的两个参数都是随机的，Y是两者的和，所以参数都是1

w1: [[-0.81131822]
 [ 1.48459876]]
w1: [[-0.67947483]
 [ 1.56272602]]
w1: [[ 1.03902912]
 [ 1.0869782 ]]
w1: [[ 1.03967905]
 [ 1.07788849]]
w1: [[ 1.03597975]
 [ 1.07681155]]
w1: [[ 1.04148865]
 [ 1.0766449 ]]
w1: [[ 1.03778934]
 [ 1.07556796]]
w1: [[ 1.04329824]
 [ 1.07540131]]
w1: [[ 1.03959894]
 [ 1.07432437]]
w1: [[ 1.04510784]
 [ 1.07415771]]
w1: [[ 1.04383802]
 [ 1.0775423 ]]
w1: [[ 1.04204392]
 [ 1.07587433]]
w1: [[ 1.04077423]
 [ 1.07925892]]
w1: [[ 1.0370748 ]
 [ 1.07818198]]
w1: [[ 1.04258382]
 [ 1.07801533]]
w1: [[ 1.0388844 ]
 [ 1.07693839]]
w1: [[ 1.0443933 ]
 [ 1.07677174]]
w1: [[ 1.040694 ]
 [ 1.0756948]]
w1: [[ 1.0369947 ]
 [ 1.07461786]]
w1: [[ 1.0425036 ]
 [ 1.07445121]]
w1: [[ 1.04123378]
 [ 1.0778358 ]]
w1: [[ 1.03753448]
 [ 1.07675886]]
w1: [[ 1.04304338]
 [ 1.07659221]]
w1: [[ 1.03934407]
 [ 1.07551527]]
w1: [[ 1.04485297]
 [ 1.07534862]]
w1: [[ 1.04358327]
 [ 1.07873321]]
w1: [[ 1.03988385]
 [ 1.07765627]]
w1: [[ 1.03618455]
 [ 1.07657933]]
w1: [[ 1.04359865]
 [ 

这是01均匀分布
help(rdm.rand)


     Create an array of the given shape and populate it with
    random samples from a uniform distribution
    over ``[0, 1)``.